In [39]:
!pip install langchain_openai
!pip install langchain_core
!pip install langchain_community
!pip install langserve
!pip install pineconedb
!pip install openai pinecone-client langchain

ERROR: Could not find a version that satisfies the requirement pineconedb (from versions: none)
ERROR: No matching distribution found for pineconedb


In [40]:
import pandas as pd
import numpy as np
import langchain_openai as lo

In [41]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [54]:
a='/content/drive/MyDrive/data.csv'
docs=pd.read_csv(a)
docs.head()

,Unnamed: 0,Reason for Call,Age,Gender,Symptoms,Disease,Diagnosis,Prescription
0,0,Severe abdominal pain in the upper right area,NA,NA,"Severe sharp pain in the upper right abdomen,...",Possible gallbladder disease,"Abdominal pain, likely related to gallbladder...",Not specified
1,1,Suspected bladder infection,30,Female,"Burning sensation during urination, frequent ...","Overactive bladder, possible urinary tract in...",Suspected urinary tract infection (UTI),"Investigations to be conducted, possible anti..."
2,2,Intense pain in right shoulder following a ru...,NA,NA,"Intense, constant sharp pain in right shoulde...",Shoulder dislocation,Shoulder dislocation confirmed by physical ex...,X-ray to confirm dislocation and check for fr...
3,3,Back pain,NA,NA,"Lower back pain for two weeks, sharp pain, wo...",Mechanical back pain,"Based on history and physical exam, likely me...","Modified activity at work, short-term bed res..."
4,4,Daughter experiencing pain and limping in the...,5,Female,"Limping, pain in the right leg and hip, diffi...","Transient synovitis, Septic arthritis, Legg-C...","Transient synovitis (primary), Septic arthrit...",Anti-inflammatories such as ibuprofen


In [58]:
import os
os.environ['OPENAI_API_KEY']="sk-proj-..."


In [59]:
import langchain_community.vectorstores as vectorstores

In [60]:
from pinecone import Pinecone, ServerlessSpec

pc = Pinecone(api_key="...")


In [61]:
from langchain.embeddings.openai import OpenAIEmbeddings
embed = OpenAIEmbeddings(
    model='text-embedding-ada-002',
    openai_api_key="sk-proj-..."
)

In [62]:
for index, row in docs.iterrows():
  string='Patient '
  if row['Age'] not in [' NA','NA',' Not specified']:
    string+=f"of Age {row['Age']},"
  if row['Gender']not in [' NA','NA',' Not specified']:
    string+=f"of Gender {row['Gender']},"
  string+=f"is having {row['Symptoms']}"
  docs.loc[index,'Summary']=string
docs.head()

,Unnamed: 0,Reason for Call,Age,Gender,Symptoms,Disease,Diagnosis,Prescription,Summary
0,0,Severe abdominal pain in the upper right area,NA,NA,"Severe sharp pain in the upper right abdomen,...",Possible gallbladder disease,"Abdominal pain, likely related to gallbladder...",Not specified,Patient is having Severe sharp pain in the up...
1,1,Suspected bladder infection,30,Female,"Burning sensation during urination, frequent ...","Overactive bladder, possible urinary tract in...",Suspected urinary tract infection (UTI),"Investigations to be conducted, possible anti...","Patient of Age 30,of Gender Female,is having..."
2,2,Intense pain in right shoulder following a ru...,NA,NA,"Intense, constant sharp pain in right shoulde...",Shoulder dislocation,Shoulder dislocation confirmed by physical ex...,X-ray to confirm dislocation and check for fr...,"Patient is having Intense, constant sharp pai..."
3,3,Back pain,NA,NA,"Lower back pain for two weeks, sharp pain, wo...",Mechanical back pain,"Based on history and physical exam, likely me...","Modified activity at work, short-term bed res...",Patient is having Lower back pain for two wee...
4,4,Daughter experiencing pain and limping in the...,5,Female,"Limping, pain in the right leg and hip, diffi...","Transient synovitis, Septic arthritis, Legg-C...","Transient synovitis (primary), Septic arthrit...",Anti-inflammatories such as ibuprofen,"Patient of Age 5,of Gender Female,is having ..."


In [63]:
docs['VectorSummary']=docs['Summary'].apply(lambda x: embed.embed_query(x))
docs.head()

,Unnamed: 0,Reason for Call,Age,Gender,Symptoms,Disease,Diagnosis,Prescription,Summary,VectorSummary
0,0,Severe abdominal pain in the upper right area,NA,NA,"Severe sharp pain in the upper right abdomen,...",Possible gallbladder disease,"Abdominal pain, likely related to gallbladder...",Not specified,Patient is having Severe sharp pain in the up...,"[0.0035097879048491916, -0.02304348576684826, ..."
1,1,Suspected bladder infection,30,Female,"Burning sensation during urination, frequent ...","Overactive bladder, possible urinary tract in...",Suspected urinary tract infection (UTI),"Investigations to be conducted, possible anti...","Patient of Age 30,of Gender Female,is having...","[-0.004388182736946255, -0.006196587596492887,..."
2,2,Intense pain in right shoulder following a ru...,NA,NA,"Intense, constant sharp pain in right shoulde...",Shoulder dislocation,Shoulder dislocation confirmed by physical ex...,X-ray to confirm dislocation and check for fr...,"Patient is having Intense, constant sharp pai...","[-0.013259365625580756, -0.0014831667639806582..."
3,3,Back pain,NA,NA,"Lower back pain for two weeks, sharp pain, wo...",Mechanical back pain,"Based on history and physical exam, likely me...","Modified activity at work, short-term bed res...",Patient is having Lower back pain for two wee...,"[0.014604454649253994, 0.003056069326240236, 0..."
4,4,Daughter experiencing pain and limping in the...,5,Female,"Limping, pain in the right leg and hip, diffi...","Transient synovitis, Septic arthritis, Legg-C...","Transient synovitis (primary), Septic arthrit...",Anti-inflammatories such as ibuprofen,"Patient of Age 5,of Gender Female,is having ...","[-0.010854082485243253, 0.010485031003581741, ..."


In [76]:
docs['Disease'].fillna('Unknown', inplace=True)
docs['Diagnosis'].fillna('Unknown', inplace=True)
docs['Prescription'].fillna('Unknown', inplace=True)

In [67]:
#docs.drop(columns=['Age','Gender'],inplace=True)
docs['Unnamed: 0']=docs['Unnamed: 0'].apply(lambda x: str(x))
docs.head()

,Unnamed: 0,Reason for Call,Age,Gender,Symptoms,Disease,Diagnosis,Prescription,Summary,VectorSummary
0,0,Severe abdominal pain in the upper right area,NA,NA,"Severe sharp pain in the upper right abdomen,...",Possible gallbladder disease,"Abdominal pain, likely related to gallbladder...",Not specified,Patient is having Severe sharp pain in the up...,"[0.0035097879048491916, -0.02304348576684826, ..."
1,1,Suspected bladder infection,30,Female,"Burning sensation during urination, frequent ...","Overactive bladder, possible urinary tract in...",Suspected urinary tract infection (UTI),"Investigations to be conducted, possible anti...","Patient of Age 30,of Gender Female,is having...","[-0.004388182736946255, -0.006196587596492887,..."
2,2,Intense pain in right shoulder following a ru...,NA,NA,"Intense, constant sharp pain in right shoulde...",Shoulder dislocation,Shoulder dislocation confirmed by physical ex...,X-ray to confirm dislocation and check for fr...,"Patient is having Intense, constant sharp pai...","[-0.013259365625580756, -0.0014831667639806582..."
3,3,Back pain,NA,NA,"Lower back pain for two weeks, sharp pain, wo...",Mechanical back pain,"Based on history and physical exam, likely me...","Modified activity at work, short-term bed res...",Patient is having Lower back pain for two wee...,"[0.014604454649253994, 0.003056069326240236, 0..."
4,4,Daughter experiencing pain and limping in the...,5,Female,"Limping, pain in the right leg and hip, diffi...","Transient synovitis, Septic arthritis, Legg-C...","Transient synovitis (primary), Septic arthrit...",Anti-inflammatories such as ibuprofen,"Patient of Age 5,of Gender Female,is having ...","[-0.010854082485243253, 0.010485031003581741, ..."


In [68]:
type(docs['Unnamed: 0'][0])

str

In [77]:
pc.create_index(
    name="disease-symptoms-gpt-4",
    dimension=len(docs['VectorSummary'][0]), # Replace with your model dimensions
    metric="cosine", # Replace with your model metric
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    )
)

In [78]:
index = pc.Index("disease-symptoms-gpt-4")
# wait a moment for connection

index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {},
 'total_vector_count': 0}

In [79]:
docs.head()
for i,j in docs.iterrows():
  index.upsert(vectors=[(j['Unnamed: 0'],j['VectorSummary'],{'text':j['Summary'],'Disease':j['Disease'],'Diagnosis':j['Diagnosis'],'Prescription':j['Prescription']})])
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 282}},
 'total_vector_count': 282}

In [80]:
from langchain.vectorstores import Pinecone
# switch back to normal index for langchain

vectorstore = Pinecone(index, embed.embed_query, "text")


/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `Pinecone` was deprecated in LangChain 0.0.18 and will be removed in 0.3.0. An updated version of the class exists in the langchain-pinecone package and should be used instead. To use it run `pip install -U langchain-pinecone` and import as `from langchain_pinecone import Pinecone`.
  warn_deprecated(
/usr/local/lib/python3.10/dist-packages/langchain_community/vectorstores/pinecone.py:68: UserWarning: Passing in `embedding` as a Callable is deprecated. Please pass in an Embeddings object instead.
  warnings.warn(


Semantic Search



In [81]:
query = "pain in the upper right abdomen, severe pain, nausea, decreased energy, decreased appetite"


vectorstore.similarity_search(
    query,  # our search query
    k=3  # return 3 most relevant docs
)

[Document(metadata={'Diagnosis': ' Abdominal pain, likely related to gallbladder issues', 'Disease': ' Possible gallbladder disease', 'Prescription': ' Not specified'}, page_content='Patient is having  Severe sharp pain in the upper right abdomen, nausea, decreased energy levels, reduced appetite'),
 Document(metadata={'Diagnosis': ' Not specified', 'Disease': ' Not specified', 'Prescription': ' Not specified'}, page_content='Patient is having  Pain in the right side of the belly, aching pain, constant pain for the past 5-6 hours, nausea, fatigue, reduced appetite'),
 Document(metadata={'Diagnosis': ' NA', 'Disease': ' NA', 'Prescription': ' NA'}, page_content='Patient is having  Abdominal pain near the right hip, crampy feeling, constipation, nausea, vomiting, feeling hot (possible fever)')]

RAG Retriver

In [92]:
from langchain.vectorstores import Pinecone as LangChainPinecone
from langchain.prompts import PromptTemplate
from openai import OpenAI
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain_core.vectorstores import VectorStoreRetriever

prompt_template='''Accept the user’s symptoms as input and provide probable diseases, diagnoses and prescription using only the information stored in the vector database. If the information is not available in the database, politely inform the user that the data is insufficient to provide a diagnosis.
    Text:
    {context}'''
PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context"]
)
# Create a RetrievalQA chain
retriever = VectorStoreRetriever(vectorstore=vectorstore)
qa_chain = RetrievalQA.from_chain_type(
    llm=ChatOpenAI(api_key="sk-proj-...",
                   model_name='gpt-4o',
                   temperature=0.0),
    chain_type="stuff",
    retriever=retriever,
    chain_type_kwargs={"prompt": PROMPT},)

In [93]:
q=' pain in the upper right abdomen, severe pain, nausea, decreased energy, decreased appetite'
answer = qa_chain.run(query=q)
print(answer)

Based on the symptoms provided, here are the probable diseases, diagnoses, and potential prescriptions:

1. **Severe sharp pain in the upper right abdomen, nausea, decreased energy levels, reduced appetite:**
   - **Probable Disease:** Gallstones or Cholecystitis (inflammation of the gallbladder)
   - **Diagnosis:** An ultrasound of the abdomen is recommended to check for gallstones or inflammation.
   - **Prescription:** Pain relievers (e.g., ibuprofen), anti-nausea medication (e.g., ondansetron), and possibly antibiotics if an infection is suspected. Surgical consultation may be necessary if gallstones are confirmed.

2. **Pain in the right side of the belly, aching pain, constant pain for the past 5-6 hours, nausea, fatigue, reduced appetite:**
   - **Probable Disease:** Appendicitis
   - **Diagnosis:** A physical examination, blood tests, and an abdominal ultrasound or CT scan are recommended to confirm appendicitis.
   - **Prescription:** Immediate medical attention is required. I